In [6]:
#Scrape Main Gird

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# Set up Chrome driver
driver = webdriver.Chrome()

# Open the URL
url = "https://coinmarketcap.com/historical/20230825/"
driver.get(url)

# Scroll down to the end of the page

height = driver.execute_script("return document.body.scrollHeight")
for i in range(0, height, 30):
    driver.execute_script(f"window.scrollTo(0,{i});")

#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)  # Wait for the page to load

# Find the table using XPath
table_xpath = "/html/body/div[1]/div[2]/div[2]/div/div[1]/div[3]/div[1]/div[3]/div/table"
table = driver.find_element(By.XPATH, table_xpath)

# Get all rows in the table
rows = table.find_elements(By.TAG_NAME, "tr")

# Initialize empty lists for data
data = []
header = []

# Extract header text
header_row = rows[0]
header_cells = header_row.find_elements(By.TAG_NAME, "th")
for cell in header_cells:
    header.append(cell.text)

# Extract data from each row
for row in rows[1:]:
    row_data = []
    cells = row.find_elements(By.TAG_NAME, "td")
    for cell in cells:
        # Get the text and link in each cell
        cell_text = cell.text
        try:
            cell_link = cell.find_element(By.TAG_NAME, "a").get_attribute("href")
        except Exception:
            cell_link = None
        row_data.append((cell_text, cell_link))
    data.append(row_data)

# Create pandas DataFrame
df = pd.DataFrame(data, columns=header)

# Save DataFrame to CSV
df.to_csv("table_data.csv", index=False)

# Close the browser
driver.quit()

In [10]:
#Scrape Historical links

import csv
import time
from selenium import webdriver

# Path to the CSV file
csv_file = 'table_data.csv'
# Number of rows to process at a time
batch_size = 200
# Delay in seconds
delay = 2

# Function to extract historical data links from a webpage
def extract_historical_links(link):
    driver = webdriver.Chrome() 
    driver.get(link)
    historical_link = driver.find_element(by='partial link text', value='See historical data')
    historical_link_url = historical_link.get_attribute('href')
    driver.quit()
    return historical_link_url

# Read the CSV file and extract the 'Name' column
names = []
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        names.append(row[1])  # Assuming 'Name' is the Second column (index 1)

# Process the names in batches
for i in range(0, len(names), batch_size):
    batch_names = names[i:i+batch_size]
    # Process each name in the batch
    for name in batch_names:
        # Extract the link from the name
        link = eval(name)[1]
        historical_link = extract_historical_links(link)
        # Save the historical link to a new CSV file
        with open('historical_links.csv', 'a', newline='') as output_file:
            writer = csv.writer(output_file)
            writer.writerow([name, historical_link])
        time.sleep(delay)

    print(f'{i+batch_size} rows processed')

print('Task completed successfully!')

200 rows processed
Task completed successfully!


In [18]:
#Scrape Github links, tags and properties

import time
import csv
import requests
import json
from bs4 import BeautifulSoup

# Path to the CSV file
csv_file = 'table_data.csv'

# Function to extract source GitHub link, tags, and additional data from a URL
def extract_data(url):
    page_source = requests.get(url).text
    soup = BeautifulSoup(page_source, "html.parser")
    data = soup.find("script", attrs={"id": "__NEXT_DATA__"}).text
    json_data = json.loads(data)
    tags = json_data['props']['pageProps']['detailRes']['detail']['tags']
    
    source_github = None
    if 'source_code' in json_data['props']['pageProps']['detailRes']['detail']["urls"]:
        source_code_urls = json_data['props']['pageProps']['detailRes']['detail']["urls"]['source_code']
        if source_code_urls:
            source_github = source_code_urls[0]
    
    tags_lst = [tag['slug'] for tag in tags]
    
    circulatingSupply = None
    totalSupply = None
    lowAllTime = None
    highAllTime = None
    
    if 'statistics' in json_data['props']['pageProps']['detailRes']['detail']:
        statistics = json_data['props']['pageProps']['detailRes']['detail']['statistics']
        circulatingSupply = statistics.get('circulatingSupply')
        totalSupply = statistics.get('totalSupply')
        lowAllTime = statistics.get('lowAllTime')
        highAllTime = statistics.get('highAllTime')
    
    return source_github, tags_lst, circulatingSupply, totalSupply, lowAllTime, highAllTime

# Read the CSV file and extract the 'Name' column
names = []
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        names.append(row[1])  # Assuming 'Name' is the second column (index 1)

# Process each URL in the 'Name' column
data_rows = []
for name in names:
    url = eval(name)[1]
    print(url)
    source_github, tags_lst, circulatingSupply, totalSupply, lowAllTime, highAllTime = extract_data(url)
    data_rows.append([name, source_github, tags_lst, circulatingSupply, totalSupply, lowAllTime, highAllTime])
    time.sleep(1)

# Save the data to a new CSV file
output_file = 'coinProp.csv'
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Source GitHub', 'Tags', 'Circulating Supply', 'Total Supply', 'Low All Time', 'High All Time'])
    writer.writerows(data_rows)

print(f'Data saved to {output_file} successfully!')

https://coinmarketcap.com/currencies/bitcoin/
https://coinmarketcap.com/currencies/ethereum/
https://coinmarketcap.com/currencies/tether/
https://coinmarketcap.com/currencies/bnb/
https://coinmarketcap.com/currencies/xrp/
https://coinmarketcap.com/currencies/usd-coin/
https://coinmarketcap.com/currencies/cardano/
https://coinmarketcap.com/currencies/dogecoin/
https://coinmarketcap.com/currencies/solana/
https://coinmarketcap.com/currencies/tron/
https://coinmarketcap.com/currencies/polkadot-new/
https://coinmarketcap.com/currencies/multi-collateral-dai/
https://coinmarketcap.com/currencies/polygon/
https://coinmarketcap.com/currencies/toncoin/
https://coinmarketcap.com/currencies/litecoin/
https://coinmarketcap.com/currencies/shiba-inu/
https://coinmarketcap.com/currencies/wrapped-bitcoin/
https://coinmarketcap.com/currencies/bitcoin-cash/
https://coinmarketcap.com/currencies/unus-sed-leo/
https://coinmarketcap.com/currencies/avalanche/
https://coinmarketcap.com/currencies/stellar/
htt

https://coinmarketcap.com/currencies/digibyte/
https://coinmarketcap.com/currencies/stargate-finance/
https://coinmarketcap.com/currencies/dao-maker/
https://coinmarketcap.com/currencies/mobilecoin/
https://coinmarketcap.com/currencies/ribbon-finance/
https://coinmarketcap.com/currencies/core-dao/
https://coinmarketcap.com/currencies/skale-network/
https://coinmarketcap.com/currencies/zel/
https://coinmarketcap.com/currencies/uma/
https://coinmarketcap.com/currencies/horizen/
https://coinmarketcap.com/currencies/galxe/
https://coinmarketcap.com/currencies/reserve-rights/
https://coinmarketcap.com/currencies/tomochain/
https://coinmarketcap.com/currencies/everscale/
https://coinmarketcap.com/currencies/standard-tokenization-protocol/
https://coinmarketcap.com/currencies/lisk/
https://coinmarketcap.com/currencies/nano/
https://coinmarketcap.com/currencies/storj/
https://coinmarketcap.com/currencies/alchemy-pay/
https://coinmarketcap.com/currencies/kyber-network-crystal-v2/
https://coinma

In [33]:
#download CSV of historical data

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

def download_file_from_url(url):
    
    # Set the download directory
    #download_dir = '/coinsCSV'

    # Configure Chrome options to set the download directory
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    #chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument(f'--download.default_directory={download_dir}')

    # Create a new Chrome driver with the configured options
    driver = webdriver.Chrome(options=chrome_options)

    # Navigate to the provided URL
    driver.get(url)

    # Click on the first button
    first_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[1]'))
    )
    first_button.click()
    
    time.sleep(2)
    
    # Scroll to the second button
    second_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[2]/ul/li[5]'))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", second_button)

    # Wait for the element to become clickable
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[2]/ul/li[5]'))
    )
    
    # Click on the second button using JavaScript
    driver.execute_script("arguments[0].click();", second_button)
    
    time.sleep(2)

    # Scroll to the third button
    third_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div/div/div[2]/span/button'))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", third_button)

    # Wait for the element to become clickable
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div/div[1]/div/div/div[2]/span/button'))
    )

    # Click on the third button using JavaScript
    driver.execute_script("arguments[0].click();", third_button)
    
    time.sleep(2)

    # Click on the download button
    download_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[2]'))
    )
    download_button.click()
    time.sleep(30)

    # Close the driver
    driver.quit()


# Example usage
url = 'https://coinmarketcap.com/currencies/bitcoin/historical-data'
download_file_from_url(url)

In [ ]:
#Scrape Tweets

import requests
import json
import time
import pandas as pd
df = pd.DataFrame(columns=["id","data"])
url = "https://api-gravity.coinmarketcap.com/gravity/v3/gravity/cdp/trending-posts/"

headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.6",
    "cache-control": "no-cache",
    "content-type": "application/json",
    "languagecode": "en",
    "platform": "web",
    "sec-ch-ua": "\"Chromium\";v=\"116\", \"Not)A;Brand\";v=\"24\", \"Brave\";v=\"116\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "sec-gpc": "1",
    "x-csrf-token": "eb9747fbef25c7dedcea6eb874df510a24601b135ed95fa88ce0293421e9accd6045ff3e3f481d5654b26cf309fb6805b6483ab2e13ce8aecac42a4fecefc5487a390ef91d7c1ac54d86914d067c1ad5ab894e41d9b5f7ae1ee591a42afdc1448c75ca097afaf2f1027e5672c38119a8492f52cae42f3cd2cd6cbed97394a84f",
    "x-request-id": "637f4a4eeadd40aca598b3acad38a21b"
}
with open("ids.txt") as f:
    for i in f:
        text = []

        data_j = {
            "cryptoId": f"{i}",
            "overView": False,
            "type": "ALL",
            "language": "en",
            "lastScore": "",
            "lastType": "trending",
            "experiment": "bffe9648-5787-4cde-8d8c-f84b841ad2d5",
            "languageCode": "en"
        }

        response = requests.post(url, headers=headers, data=json.dumps(data_j), verify=False)
        data = response.json()

        for tweet in data['data']['tweetDTOList']:
            text.append(tweet['textContent'])
        last_score = data['data']['tweetDTOList'][-1]['trendingScore']

        time.sleep(2)  
        data_j = {
            "cryptoId": f"{i}",
            "overView": False,
            "type": "ALL",
            "language": "en",
            "lastScore": f"{last_score}",
            "lastType": "trending",
            "experiment": "bffe9648-5787-4cde-8d8c-f84b841ad2d5",
            "languageCode": "en"
        }

        response = requests.post(url, headers=headers, data=json.dumps(data_j), verify=False)
        data = response.json()

        for tweet in data['data']['tweetDTOList']:
            text.append(tweet['textContent'])
        df.loc[len(df)] = [i,text]
